In [1]:
import numpy as np

NOTE: Throughout this note book. The glasses will be numbered from $1$ to $n$ for the player (as in the rubric), but numbered from $0$ to $n-1$ for the implementation of the code as is the coonvention used for Python.

In [4]:
def neighbours(circle):
    # return the indexes of the neighboours of each glass
    n = len(circle)
    neigh = [[n-1, 1]]
    for i in range(1, n-1): # For glass number 2, ..., n-1
        adj = [i-1, i+1]
        neigh.append(adj)
    neigh.append([n-2, 0])
    return neigh

In [5]:
# Check logic for neighbours: 
arr = [0, 0.2, 0.3, 0.1, 0.1]

print("Neighbours using Python indexing:", neighbours(arr))

print("The neighbours of glass 5 are glass number", neighbours(arr)[5-1][0]+1, "and", neighbours(arr)[5-1][1]+1)

Neighbours using Python indexing: [[4, 1], [0, 2], [1, 3], [2, 4], [3, 0]]
The neighbours of glass 5 are glass number 4 and 1


In [6]:
def spill(Circling):
    # Function that checks for any spillage
    return not all(i <= 1 for i in Circling)

In [7]:
# Cehck logic for spill:
spill([0, 0.2, 1.2, 0.7])

True

In [8]:
def Ali(circle):
    # Given the current state of the circle,
    # Prompt Ali to make a move
    print("Ali, please enter the amount of pints of water you would like to pour for each glass!")
    amount = [float(x) for x in input('Amount (pints): ').split()]
    
    # Check for invalid entries
    if abs(sum(amount) - 0.5) > 10E-6:
        print("Invalid Entry! The sum of the amount should be 0.5!")
        print("Please try again.")
        return 0
    
    for i in amount:
        if i < 0:
            print("Invalid Entry! Ali cannot pour water out!")
            print("Please try again.")
            return 0
    
    for i in range(len(circle)):
        circle[i] += amount[i]
    
    return circle

In [9]:
# Check logic for Ali:
arr = [0, 0, 0, 0.4]
Ali(arr)

Ali, please enter the amount of pints of water you would like to pour for each glass!
Amount (pints): 0.2 0.2 0.1 0


[0.2, 0.2, 0.1, 0.4]

In [10]:
Ali(arr)

Ali, please enter the amount of pints of water you would like to pour for each glass!
Amount (pints): 0.1 0.1 0.1 0.2


[0.30000000000000004, 0.30000000000000004, 0.2, 0.6000000000000001]

In [11]:
def Beth(circle):
    # Beth bot with strategy as defined in the rubric
    check = [] # Initialise the list to store the index of any glass with more than 0.5 pint of water
    for i in range(len(circle)):
        if circle[i] >= 0.5:
            check.append(i)
    if len(check) == 2: # Thanks to the CheckMate function below, we may assume they are adjacent
        circle[check[0]] = 0
        circle[check[1]] = 0
    elif len(check) == 1:
        left, right = neighbours(circle[check[0]])[0], neighbours(circle[check[0]])[1]
        if left > right:
            circle[left] = 0
        else:
            circle[right] = 0
        circle[check[0]] = 0
    else:
        ttl = []
        for i in range(len(circle)):
            adsum = circle[i] + circle[neighbours(circle)[i][1]]
            ttl.append(adsum)
        idx = ttl.index(max(ttl))
        circle[idx] = 0
        circle[neighbours(circle)[idx][1]] = 0
    
    return circle
        
    

In [12]:
# Check logic for Beth
c = [0.2, 0.1, 0.1, 0.2]
Beth(c)

[0, 0.1, 0.1, 0]

In [13]:
def CheckMate(circle):
    # Check if Beth is check mated
    checks = []
    for i in range(len(circle)):
        if circle[i] > 0.5:
            checks.append(i)
            
    
    if len(checks) > 2:
        return True
    elif len(checks) == 2:
        idx1, idx2 = checks[0], checks[1]
        if idx2 == neighbours(circle)[idx1][0] or idx2 == neighbours(circle)[idx1][1]: # Check if they are neighbours
            return False
        else:
            return True
    else:
        return False
        

In [14]:
# Check logic for CheckMate
arr = [0, 0.51, 0.2, 0.6, 0.1]
arr2 = [0, 0, 0.51, 0.50001, 0.2]
print(CheckMate(arr))
print(CheckMate(arr2))

True
False


In [15]:
def EqStraight(pos1, pos2):
    # Check if two positions are equal without reflection
    ret = False
    n = len(pos1)
    for i in range(n):
        equal = 0
        for j in range(n):
            if pos1[(j + i)%n] == pos2[j]:
                equal += 1
        if equal == n:
            ret = True

    return ret

def EqReflect(pos1, pos2):
    # Check if two positions are equal when reflected
    list(pos1).reverse()
    return EqStraight(pos1, pos2)

def Equal(pos1, pos2):
    test1 = EqStraight(pos1, pos2)
    test2 = EqReflect(pos1, pos2)
    return test1 or test2

In [16]:
p1 = [0, 0, 0.23, 0.68]
p2 = [0, 0.68, 0.23, 0]
Equal(p1, p2)

False

In [17]:
def game(n):
    # Start game with Ali, where Beth is a bot as mentioned in the entry phase
    # Initialise the glasses in circular arrangement
    circle = np.zeros(n)
    #Initialise history array which stores each position after every turn
    history = [circle.copy()]
    print(history)
    # Initialise neighbours list to keep track of the neighbours of each glass
    # adj = neighbours(circle)
    
    Play = True
    Winner = 0
    while Play: # For each turn: 
        # Ali goes first
        print(circle)
        print("Ali's turn!")
        new = Ali(circle)
        
        # Check if Ali won
        if CheckMate(circle):
            Winner = 1
            Play = False
        
        print(circle)
        history.append(circle.copy())
        print(history)
        # Beth's move, but Beth is a bot with predefined strategies as in the rubric
        print("Beth's turn!")
        circle = Beth(circle)
        
        print(circle)
        history.append(circle.copy())
        print(history)
        # Check if Beth won
        for i in range(len(history) - 1):
            if Equal(circle, history[i]):
                print(history)
                Winner = 2
                Play = False
                
    # Delivering the final message when the game ends
    Players = ["Ali", "Beth"]
    print("The winner is ...")
    print(Winner)
    return Players[Winner-1]

game(3)

[array([0., 0., 0.])]
[0. 0. 0.]
Ali's turn!
Ali, please enter the amount of pints of water you would like to pour for each glass!
Amount (pints): 0.2 0.1 0.2
[0.2 0.1 0.2]
[array([0., 0., 0.]), array([0.2, 0.1, 0.2])]
Beth's turn!
[0.  0.1 0. ]
[array([0., 0., 0.]), array([0.2, 0.1, 0.2]), array([0. , 0.1, 0. ])]
[0.  0.1 0. ]
Ali's turn!
Ali, please enter the amount of pints of water you would like to pour for each glass!
Amount (pints): 0.2 0.1 0.2
[0.2 0.2 0.2]
[array([0., 0., 0.]), array([0.2, 0.1, 0.2]), array([0. , 0.1, 0. ]), array([0.2, 0.2, 0.2])]
Beth's turn!
[0.  0.  0.2]
[array([0., 0., 0.]), array([0.2, 0.1, 0.2]), array([0. , 0.1, 0. ]), array([0.2, 0.2, 0.2]), array([0. , 0. , 0.2])]
[0.  0.  0.2]
Ali's turn!
Ali, please enter the amount of pints of water you would like to pour for each glass!
Amount (pints): 0.23 0.23 0.06
Invalid Entry! The sum of the amount should be 0.5!
Please try again.
[0.  0.  0.2]
[array([0., 0., 0.]), array([0.2, 0.1, 0.2]), array([0. , 0.1, 0

'Beth'